# Introduction to Quantities

Provided by AstroPy

* [documentation](http://docs.astropy.org/en/stable/units/)
* [tutorial](http://www.astropy.org/astropy-tutorials/Quantities.html)


## SunPy has a policy to use units everywhere!

## Creation

In [1]:
from astropy import units as u

In [2]:
length = 26.2 * u.meter
length

<Quantity 26.2 m>

In [3]:
lengths = [1,2] * u.Angstrom
lengths

<Quantity [ 1., 2.] Angstrom>

In [4]:
u.Quantity([1,2], 'Angstrom')

<Quantity [ 1., 2.] Angstrom>

In [5]:
print("The value is {0} and the unit is {1}".format(str(length.value), str(length.unit)))

The value is 26.2 and the unit is m


Quantities behave the same as numpy arrays

In [6]:
lengths * length

<Quantity [ 26.2, 52.4] Angstrom m>

## Units stop you from doing stupid things!

In [7]:
5 * u.arcmin + 12 * u.meter

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

Quantities can be converted to other units systems or factors by using `to()`

In [8]:
# Convert it to: km, lyr
print(length.to(u.km))
print(length.to('lightyear'))

0.0262 km
2.7693421851444923e-15 lyr


Quantities can also be combined, for example to measure speed

In [9]:
# calculate a speed
time = 15 * u.minute
speed = length / time
print(speed)

1.7466666666666666 m / min


In [10]:
# decompose it
print(speed.decompose())

0.02911111111111111 m / s


Imperial (english) units are also available

In [11]:
from astropy.units import imperial
print(speed.to(imperial.mile/u.hour))

0.0651197009464726 mi / h


## Unitless

In [12]:
# no units
nounits = 20. * u.cm / (1. * u.m)
print(nounits)
print(nounits.decompose())

20.0 cm / m
0.2


What happen if we add a number to this?

In [13]:
# arithmetic with no units
nounits + 3

<Quantity 3.2>

In [14]:
type(nounits + 3)

astropy.units.quantity.Quantity

In [15]:
nounits.decompose().unit

Unit(dimensionless)

# Constants

In [16]:
from sunpy.sun import constants

In [17]:
print(constants.au)

  Name   = Astronomical Unit
  Value  = 149597870700.0
  Uncertainty  = 0.0
  Unit  = m
  Reference = IAU 2012 Resolution B2


In [18]:
scale = (np.tan(1 * u.arcsec) * constants.au).to('km')
scale

<Quantity 725.2709438078364 km>

## Equivalencies

Some conversions are not done by a conversion factor as between miles and kilometers, for example converting between wavelength and frequency.

In [19]:
# converting spectral quantities
(656.281 * u.nm).to(u.Hz) # Fails because they are not compatible

UnitConversionError: 'nm' (length) and 'Hz' (frequency) are not convertible

In [20]:
# but doing it right
(656.281 * u.nm).to(u.Hz, equivalencies=u.spectral())

<Quantity 456805024067434.5 Hz>

In [21]:
(1 * u.keV).to('MK', equivalencies=u.equivalencies.temperature_energy())

<Quantity 11.60451930280894 MK>

[Other built-in equivalencies](http://docs.astropy.org/en/stable/units/index.html#module-astropy.units.equivalencies)

[Define your own equivalency](http://astropy.readthedocs.io/en/stable/units/equivalencies.html#writing-new-equivalencies)

(from_unit, to_unit, forward, backward)

In [22]:
plate_scale = [(u.arcsec, u.km, lambda x: scale.value * x, lambda x:  x / scale.value)]

In [23]:
(1 * u.arcmin).to('Mm', equivalencies=plate_scale)

<Quantity 43.51625662847018 Mm>

In [24]:
(constants.radius).to('arcsec', equivalencies=plate_scale)

<Quantity 958.9629998803285 arcsec>

## Creating functions with quantities as units

We want to have functions that contain the information of the untis, and with them we can be sure that we will be always have the *right* result.

In [25]:
# Create a function for the Kinetic energy
@u.quantity_input(mass=u.kg, speed=u.m/u.s)
def kinetic(mass, speed):
    return (0.5 * mass * speed ** 2).to(u.joule)

In [26]:
# run with and without units
kinetic(5, 10) # Fails! it doesn't specify the units.

TypeError: Argument 'mass' to function 'kinetic' has no 'unit' attribute. You may want to pass in an astropy Quantity instead.

In [27]:
kinetic(5 * u.kg, 100 * u.m/u.hour)

<Quantity 0.0019290123456790125 J>

## Create your own units

Some times we want to create our own units:

In [28]:
# Create units for a laugh scale
det = u.def_unit('detector')
u.add_enabled_units(det)

In [29]:
u.Quantity(7, 'W/detector')

<Quantity 7.0 W / detector>

In [30]:
u.add_enabled_units(u.def_unit('nuke', 63 * u.TJ))

In [31]:
(10**27 * u.erg).to('nuke')

<Quantity 1587301.5873015875 nuke>